In [1]:
#importing libraries
#using torch rather than the keras
import numpy as np
import cv2
import os
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.io 
import pandas as pd

In [2]:
def load_images(image_path):
    '''
        goes through all the files in folder, converts them to 
        greyscale and processes them to be used for the models
    '''
    images = []
    image_names = []
    for filename in os.listdir(image_path):
        if filename.lower().endswith(('.jpg')):
            img_path = os.path.join(image_path, filename)
            img = Image.open(img_path).convert("L")#converting image to greyscale
            img = img.convert("L")
            img = np.array(img)#converting from pillow obj to NumPy array
            img_3ch = np.stack([img, img, img], axis=-1)#duplicating channels
            img_3ch = Image.fromarray(img_3ch.astype(np.uint8)) #converting back to pillow obj
            images.append(img_3ch)
            image_names.append(filename)
    return images, image_names

In [4]:
def get_transform():
    '''
        processes image by resizing it to 224x224, converting it 
        to a PyTorch tensor (data structure used in deep learning), and 
        normalizes the image using ImageNet mean and std values

    '''
    return transforms.Compose([transforms.Resize((224, 224)),
                               transforms.ToTensor(),
                               transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                    std=[0.229, 0.244, 0.225])
    ])

In [5]:
def load_model(model_name):
    if model_name == 'vgg16':
        model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
    elif model_name == 'resnet50':
        model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    else:
        raise ValueError("Not an available model.")
    model.eval()#turns off batch normalization updates
    #returning feature vectors
    return torch.nn.Sequential(*(list(model.children())[:-1]))#returns models without final class. layer

In [6]:
def extract_features(model, images):
    transform = get_transform()
    features = []
    for img in images:
        img_t = transform(img).unsqueeze(0)#adding a batch dimension
        with torch.no_grad():#disables gradient computation
            feat = model(img_t)
            feat = feat.view(feat.size(0), -1)#making output a 1D vector
            features.append(feat.numpy()[0]) #changing it into a numpy array and appending to features
    return np.array(features)

In [7]:
def compute_pairwise_similarity(features, image_names):
    sim_matrix = cosine_similarity(features)
    df = pd.DataFrame(sim_matrix, index=image_names, columns=image_names)
    return df, sim_matrix

In [8]:
def save_to_csv(df, output_path):
    df.to_csv(output_path, index=True)

In [9]:
def generate_heatmap(df, output_path):
    plt.figure(figsize=(10, 8))
    sns.heatmap(df, annot=False, cmap="coolwarm", xticklabels=True, yticklabels=True)
    plt.title("Pairwise Image Similarity Heatmap", fontsize=14)
    plt.tight_layout()
    plt.savefig(output_path, dpi=300)
    plt.close()

In [ ]:
def image_comparison(model_name, image_path):
    images, image_names = load_images(image_path)
    model = load_model(model_name)
    features = extract_features(model, images)
    df, sim_matrix = compute_pairwise_similarity(features, image_names)
    save_to_csv(df, 'pairwise_results.csv')
    generate_heatmap(df, 'pairwise_heatmap.png')
    labels = extract_labels(image_names)
    #labels = image_names
    accuracy = category_similarity_accuracy(sim_matrix, labels)
    print(f"Category Similarity Accuracy: {accuracy:.3f}")

In [11]:
def extract_labels(image_names):
    return np.array([name.split("_")[1][:4] for name in image_names])

In [12]:
def category_similarity_accuracy(sim_matrix, labels):
    same_class_sims = []
    diff_class_sims = []
    n = len(labels)
    for i in range(n):
        for j in range(i+1, n):
            if labels[i] == labels[j]:
                same_class_sims.append(sim_matrix[i, j])
            else:
                diff_class_sims.append(sim_matrix[i, j])
    threshold = np.median(diff_class_sims)
    correct = sum(sim > threshold for sim in same_class_sims)
    accuracy = correct / len(same_class_sims)
    return accuracy

In [15]:
print("Loading in images")
image_path = r"C:\Users\zobes\OneDrive\SummerResearch\grey_images"
print("Completed")
#image_path = input("Enter the folder path with grey images: ")
model_name = input("Enter the method you would like to use for image comparison (vgg16 or resnet50): ")
print("preparing model")
image_comparison(model_name, image_path)
print("outputting comparisons")

Loading in images
Completed
preparing model
Category Similarity Accuracy: 0.972
outputting comparisons
